<a href="https://colab.research.google.com/github/tiennvuit/CS106.K21.KHTN/blob/master/18521489_NguyenVanTien_PolicyIteration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cài đặt và đánh giá thuật toán Policy Iteration

## Cài đặt thuật toán toán

In [0]:
import numpy as np
import gym
import time

In [0]:
"""
    Reference: https://medium.com/@m.alzantot/deep-reinforcement-learning-demysitifed-episode-2-policy-iteration-value-iteration-and-q-978f9e89ddaa
"""
import numpy as np
import gym
from gym import wrappers

def run_episode(env, policy, gamma = 1.0, render = False):
    """ Evaluates policy by using it to run an episode and finding its
    total reward.
    args:
    env: gym environment.
    policy: the policy to be used.
    gamma: discount factor.
    render: boolean to turn rendering on/off.
    returns:
    total reward: real value of the total reward recieved by agent under policy.
    """
    obs = env.reset()
    total_reward = 0
    step_idx = 0
    while True:
        if render:
            env.render()
        obs, reward, done , _ = env.step(int(policy[obs]))
        total_reward += (gamma ** step_idx * reward)
        step_idx += 1
        if done:
            break
    return total_reward


def evaluate_policy(env, policy, gamma = 1.0,  n = 100):
    """ Evaluates a policy by running it n times.
    returns:
    average total reward
    """
    scores = [
            run_episode(env, policy, gamma = gamma, render = False)
            for _ in range(n)]
    return np.mean(scores)

def extract_policy(v, gamma = 1.0):
    """ Extract the policy given a value-function """
    policy = np.zeros(env.nS)
    for s in range(env.nS):
        q_sa = np.zeros(env.action_space.n)
        for a in range(env.action_space.n):
            for next_sr in env.P[s][a]:
                # next_sr is a tuple of (probability, next state, reward, done)
                p, s_, r, _ = next_sr
                q_sa[a] += (p * (r + gamma * v[s_]))
        policy[s] = np.argmax(q_sa)
    return policy


def value_iteration(env, gamma = 1.0):
    """ Value-iteration algorithm """
    v = np.zeros(env.nS)  # initialize value-function
    max_iterations = 100000
    eps = 1e-20
    for i in range(max_iterations):
        prev_v = np.copy(v)
        for s in range(env.nS):
            q_sa = [sum([p*(r + prev_v[s_]) for p, s_, r, _ in env.P[s][a]]) for a in range(env.nA)] 
            v[s] = max(q_sa)
        if (np.sum(np.fabs(prev_v - v)) <= eps):
            print ('Value-iteration converged at iteration# %d.' %(i+1))
            break
    return v

In [3]:
env_name  = 'FrozenLake8x8-v0'
gamma = 0.8
env = gym.make(env_name)

start_time = time.time()
optimal_v = value_iteration(env, gamma)
policy = extract_policy(optimal_v, gamma)
print("Execute time: {}".format(time.time()-start_time))

policy_score = evaluate_policy(env, policy, gamma, n=1000)
print('Policy average score = ', policy_score)

Value-iteration converged at iteration# 2357.
Execute time: 1.8212459087371826
Policy average score =  5.146928376457513e-05


## Đánh giá thuật toán Policy Iteration

[Đây](https://stackoverflow.com/questions/37370015/what-is-the-difference-between-value-iteration-and-policy-iteration) là cuộc bàn luận về câu hỏi phân biệt giữa Value Iteration và Policy Iteration trên diễn đàng stackoverflow, dựa vào cuộc bàn luận này và kiến thức học được em có một số so sánh về 2 thuật toán được trình bày như sau:



*   Thuật toán Policy Iteration gồm có hai giai đoạn được lặp lại: policy evaluation và policy improvement; việc lặp lại này dừng khi chiến lược tìm được hội tụ. Trong khi đó, thuật toán Value Iteration gồm 2 giai đoạn được lặp lại là **finding optimal value function** và **policy extraction** và không hề có việc lặp lại vì một khi thuật toán đạt được hàm tối ưu thì chiến thuật được suy ra từ nó sẽ cũng tối ưu.

*   Phân tích thêm một chút về giai đoạn **finding optimal value function** thì ta có thể nhận thấy nó chính là việc kết hợp của *policy improvement* và một phần *policy evaluation*.


*Về mặt tốc độ thì thuật toán Policy Iteration thường sẽ nhanh hơn thuật toán Value Iteration vì một chiến lược hội tụ nhanh hơn hàm giá trị.*


![The comparision between two algorithms](https://i.stack.imgur.com/wGuj5.png)